# OPENAI

In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
# -----------------------------------------------------   

def display_summaryOPENAI(system_prompt, question):
    # ................
    # Cargamos las variables del fichero .env
    load_dotenv()
    api_key = os.getenv('OPENAI_API_KEY')
    
    # Check the key
    if not api_key:
        print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
    elif not api_key.startswith("sk-proj-"):
        print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
    elif api_key.strip() != api_key:
        print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
    
    openai = OpenAI()
    # Si esto no funciona, prueba con el menú Kernel >> Reiniciar Kernel y borrar las salidas de todas las celdas, luego ejecuta las celdas desde la parte superior de este cuaderno hacia abajo.
    # Si TODAVÍA no funciona (¡qué horror!), consulta el cuaderno de resolución de problemas o prueba la siguiente línea:
    # openai = OpenAI(api_key="your-key-here-starting-sk-proj-")
    # ................
    payload = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    # ................
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = payload
    )    
    summary = response.choices[0].message.content
    # ................
    # Salida clara en Jupyter, usando markdown
    display(Markdown(summary))

# -----------------------------------------------------   
system_prompt = "You are an expert developer with deep knowledge in C#, Python, Mathematics, Statistics, and Finance. \
Your expertise allows you to solve complex technical problems, analyze data, and implement solutions using these skills effectively. \
When answering, be precise, clear, and provide detailed explanations when needed. \
Offer solutions that are practical, efficient, and suitable for real-world applications in the domains mentioned above."

question = """
Create a function in C# to connect to an api with those parameters. The api is OpenAI api and need all functionallity:
    api_key = "werqiwenqowienfqowefmnqwefnoi340928390jrfnwekf"
    openai = OpenAI()
    system_prompt = "You are an expert developer"
    question = "Who is Python"
    payload = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = payload
    )    
    summary = response.choices[0].message.content
"""

display_summaryOPENAI(system_prompt, question)

To create a function in C# that connects to the OpenAI API with the specified parameters, we'll need to use the `HttpClient` class to make HTTP requests. Below is a complete example that demonstrates how to set up a function to interact with the OpenAI API, including handling the necessary authentication, building the payload, and parsing the response.

First, ensure you have the necessary NuGet package for JSON serialization. You can use `Newtonsoft.Json` for this purpose. You can install it via NuGet Package Manager Console:

```bash
Install-Package Newtonsoft.Json
```

Here's the complete C# function to connect to the OpenAI API:

```csharp
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Threading.Tasks;
using Newtonsoft.Json;

public class OpenAI
{
    private readonly string apiKey;
    private readonly HttpClient httpClient;

    public OpenAI(string apiKey)
    {
        this.apiKey = apiKey;

        httpClient = new HttpClient();
        httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", apiKey);
        httpClient.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
    }

    public async Task<string> GetOpenAIResponseAsync(string systemPrompt, string userQuestion)
    {
        var payload = new[]
        {
            new { role = "system", content = systemPrompt },
            new { role = "user", content = userQuestion }
        };

        var requestBody = new
        {
            model = "gpt-4o-mini",
            messages = payload
        };

        var jsonRequestBody = JsonConvert.SerializeObject(requestBody);
        var content = new StringContent(jsonRequestBody, Encoding.UTF8, "application/json");

        try
        {
            var response = await httpClient.PostAsync("https://api.openai.com/v1/chat/completions", content);
            response.EnsureSuccessStatusCode();

            var jsonResponse = await response.Content.ReadAsStringAsync();
            dynamic responseObject = JsonConvert.DeserializeObject(jsonResponse);
            string summary = responseObject.choices[0].message.content;

            return summary;
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error calling OpenAI API: {ex.Message}");
            return null;
        }
    }
}

public class Program
{
    public static async Task Main(string[] args)
    {
        string apiKey = "werqiwenqowienfqowefmnqwefnoi340928390jrfnwekf";
        OpenAI openai = new OpenAI(apiKey);
        
        string systemPrompt = "You are an expert developer";
        string question = "Who is Python";

        string response = await openai.GetOpenAIResponseAsync(systemPrompt, question);
        Console.WriteLine(response);
    }
}
```

### Explanation:

1. **HttpClient Setup**: The `OpenAI` class contains an `HttpClient` instance that is configured with the necessary authorization header for the API key.

2. **Method**: The `GetOpenAIResponseAsync` method constructs the payload as per the API requirements, serializes it to JSON using `JsonConvert.SerializeObject`, and sends it as a `POST` request to the OpenAI completions endpoint.

3. **Response Handling**: The response from the API call is deserialized from JSON into a dynamic object, from which the answer is extracted.

4. **Error Handling**: A `try-catch` block captures any exceptions that may occur during the HTTP request and logs them to the console.

5. **Demonstration**: The `Program.Main` method shows how to use the `OpenAI` class to get a response from the API.

### Usage:
To run this code, make sure you replace the `apiKey` variable value with your actual OpenAI API key. You can incorporate this code into any C# project and test it to get the result from the OpenAI API.

### Important Note:
The model name `"gpt-4o-mini"` might not be valid if it’s not an officially released OpenAI model. Always check the latest OpenAI model names in the official documentation.

# WEB SCRAPING

In [8]:
import requests
from bs4 import BeautifulSoup
# ----------------------------------------------------- 

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """
    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
# ----------------------------------------------------- 

website = Website("https://platzi.com")

user_prompt = f"Estás viendo un sitio web titulado {website.title}"
user_prompt += "\nEl contenido de este sitio web es el siguiente; \
proporciona un breve resumen de este sitio web en formato Markdown. \
Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
user_prompt += website.text

system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

display_summaryOPENAI(system_prompt, user_prompt)

# Resumen de Platzi

Platzi es una plataforma de educación en línea enfocada en la tecnología y el aprendizaje profesional, que cuenta con más de 5 millones de estudiantes y 3,000 empresas que utilizan sus servicios. Ofrece una amplia variedad de cursos en diferentes áreas, incluyendo:

- **Data Science e Inteligencia Artificial**
- **Ciberseguridad**
- **Desarrollo Web**
- **Marketing Digital**
- **Finanzas e Inversiones**
- **Blockchain y Web3**
- **Diseño Gráfico**

## Características Destacadas

- Inicio de cursos sin costo.
- Rutas de aprendizaje personalizadas que guían desde nivel principiante hasta profesional.
- Evaluaciones interactivas para medir el progreso.
- Recursos descargables para aprendizaje offline.
- Certificaciones oficiales en colaboración con entidades como ETS y CompTIA.

## Profesores

Los cursos son impartidos por expertos de la industria, como Carli Florida (Python), Anibal Rojas (ChatGPT para Empresas), y Carolina Castañeda (Bases de Datos con SQL).

## Oportunidades para Empresas

Platzi también ofrece opciones de formación y desarrollo profesional para empresas, permitiendo organizar el aprendizaje de los equipos y siguiendo su progreso.